# **Worksapce set up**

In [15]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import zipfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib


In [17]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [18]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Clustering**

In [19]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               1152      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11,521
Trainable params: 11,521
Non-trainable params: 0
_________________________________________________________________
Saved baseline model to: /tmp/tmpnmi56pgz.h5


In [20]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

clustered_model = cluster_weights(base_model, **clustering_params)

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
clustered_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
clustered_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_dense_8 (ClusterWei  (None, 128)              2192      
 ghts)                                                           
                                                                 
 cluster_dense_9 (ClusterWei  (None, 64)               16464     
 ghts)                                                           
                                                                 
 cluster_dense_10 (ClusterWe  (None, 32)               4144      
 ights)                                                          
                                                                 
 cluster_dense_11 (ClusterWe  (None, 1)                81        
 ights)                                                          
                                                                 
Total params: 22,881
Trainable params: 11,585
Non-trai

In [21]:
clustered_model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=10)
clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(clustered_model, clustered_keras_file, include_optimizer=False)
print('Saved clustered Keras model to:', clustered_keras_file)

Epoch 1/10
46/46 [==============================] - 2s 13ms/step - loss: 0.5319 - accuracy: 0.7457 - val_loss: 0.5841 - val_accuracy: 0.7370
Epoch 2/10
46/46 [==============================] - 0s 8ms/step - loss: 0.5264 - accuracy: 0.7522 - val_loss: 0.5769 - val_accuracy: 0.7435
Epoch 3/10
46/46 [==============================] - 0s 9ms/step - loss: 0.5171 - accuracy: 0.7522 - val_loss: 0.5662 - val_accuracy: 0.7403
Epoch 4/10
46/46 [==============================] - 0s 9ms/step - loss: 0.5119 - accuracy: 0.7587 - val_loss: 0.5582 - val_accuracy: 0.7500
Epoch 5/10
46/46 [==============================] - 0s 8ms/step - loss: 0.5062 - accuracy: 0.7587 - val_loss: 0.5549 - val_accuracy: 0.7565
Epoch 6/10
46/46 [==============================] - 0s 10ms/step - loss: 0.5058 - accuracy: 0.7543 - val_loss: 0.5601 - val_accuracy: 0.7403
Epoch 7/10
46/46 [==============================] - 0s 9ms/step - loss: 0.4980 - accuracy: 0.7543 - val_loss: 0.5458 - val_accuracy: 0.7630
Epoch 8/10
46/46 [

Saved clustered Keras model to: /tmp/tmppfoe2cv8.h5


# **Quantization and conversion to tflite**

In [22]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(clustered_model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_clustered.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7p499q5i/assets


INFO:tensorflow:Assets written to: /tmp/tmp7p499q5i/assets


14736

In [23]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the FULLY_CONNECTED op takes
tensor #0 and tensor #5 and tensor #4 as input and produces tensor #9 as output.

Subgraph#0 main(T#0) -> [T#13]
  Op#0 FULLY_CONNECTED(T#0, T#5, T#4) -> [T#9]
  Op#1 FULLY_CONNECTED(T#9, T#6, T#3) -> [T#10]
  Op#2 FULLY_CONNECTED(T#10, T#7, T#2) -> [T#11]
  Op#3 FULLY_CONNECTED(T#11, T#8, T#1) -> [T#12]
  Op#4 LOGISTIC(T#12) -> [T#13]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(bias_3) shape:[1], type:FLOAT32 RO 4 bytes
  T#2(bias_2) shape:[32], type:FLOAT32 RO 128 bytes
  T#3(bias_1) shape:[64], type:FLOAT32 RO 256 bytes
  T#4(bias) shape:[128], type:FLOAT32 RO 512 bytes
  T#5(sequential_2/dense_8/MatMul) shape:[128, 8], type:INT8 RO 1024 bytes
  T#6(sequential_2/dense_9/MatMul) shape:[64, 128], type:INT8 RO 8192 bytes
  T#7(sequen

In [24]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.7824675324675324


# **Conversion to C array**

In [25]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [26]:
# Write TFLite model to a C source (or header) file
with open("ANN_clustered" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_clustered"))

# **Size Comparison**

In [27]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [28]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size('ANN_clustered.tflite')))

Size of gzipped baseline Keras model: 45026.00 bytes
Size of gzipped clustered Keras model: 10839.00 bytes
Size of gzipped clustered TFlite model: 8510.00 bytes
